# Brains4Buildings interactive inverse grey-box analysis pipeline

This Jupyter Labs notebook can be used to interactively test the Brains4Buildings inverse grey-box analysis pipeline.
Don't forget to install the requirements listed in [requirements.txt](../requirements.txt) first!



## Setting the stage

First several imports and variables need to be defined


### Imports and generic settings

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from tqdm.notebook import tqdm

from gekko import GEKKO

import sys
sys.path.append('../data/')
sys.path.append('../view/')
sys.path.append('../analysis/')

# usually, two decimals suffice for displaying DataFrames (NB internally, precision may be higher)
pd.options.display.precision = 2

%load_ext autoreload

from preprocessor import Preprocessor
from inversegreyboxmodel import Learner

%matplotlib inline
%matplotlib widget

from plotter import Plot

import logging
logging.basicConfig(level=logging.ERROR, 
                    format='%(asctime)s %(levelname)-8s %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    filename='log_b4b.txt',
                   )

### Load Data from Virtual Rooms

In [ ]:
%%time
# Prerequisite: for this example to work, you need to have the b4b_raw_properties.parquet, located e.g. in the ../data/ folder.
# One way to get this is to run B4BExtractionBackup.ipynb first
df_prop = pd.read_parquet('../data/b4b_virtualrooms_raw_properties.parquet', engine='pyarrow')

#sorting the DataFrame index is needed to get good performance on certain filters
#this guarding code to check whether DataFramews are properly sorted
if not df_prop.index.is_monotonic_increasing:
    print('df_prop needed index sorting')
    df_prop = df_prop.sort_index()  

In [ ]:
df_prop.index.unique(level='id')

In [ ]:
df_prop

### Convert Virtual Room Property data to preprocessed data

In [ ]:
# for virtual room data (with no noise and measurement errors), preprocessing is trivial; just unstack source and prefix source to property name
df_prep = Preprocessor.unstack_prop(df_prop)

In [ ]:
df_prep.info()

In [ ]:
df_prep

In [ ]:
%autoreload 2
units_to_mathtext = property_types = {
    'degC' : r'$°C$',
    'ppm' : r'$ppm$',
    '0' : r'$[-]$',
    'bool': r'$0 = False; 1 = True$',
    'p' : r'$persons$'
}

In [ ]:
#Plot all properties from all sources for all ids
# Plot.dataframe_preprocessed_plot(df_prep, units_to_mathtext)

## Learn parameters using inverse grey-box analysis

Most of the heavy lifting is done by the `learn_room_parameters()` function, which again uses the [GEKKO Python](https://machinelearning.byu.edu/) dynamic optimization toolkit.

In [ ]:
%%time 
%autoreload 2

# learn the model parameters and write results to a dataframe

df_results, df_prep = Learner.learn_room_parameters(df_prep, 
                                                    col_co2__ppm = 'model_co2__ppm', 
                                                    col_occupancy__p = 'model_occupancy__p', 
                                                    col_valve_frac__0 = 'model_valve_frac__0',
                                                    learn_period__d = 7, 
                                                    req_col = ['model_co2__ppm','model_occupancy__p', 'model_valve_frac__0'],
                                                    sanity_threshold_timedelta = timedelta(hours=24),
                                                    learn_infilt__m2 = True,
                                                    learn_valve_frac__0 = False,
                                                    learn_occupancy__p = False,
                                                    ev_type=2)

### Result Visualization

In [ ]:
df_results

In [ ]:
df_prep

In [ ]:
%autoreload 2
units_to_mathtext = property_types = {
    'degC' : r'$°C$',
    'ppm' : r'$ppm$',
    '0' : r'$[-]$',
    'bool': r'$0 = False; 1 = True$',
    'p' : r'$persons$'
}

In [ ]:
#Plot all properties from all sources for all ids
Plot.dataframe_preprocessed_plot(df_prep, units_to_mathtext)